# Wrangle Train Demand Data

In [18]:
import pandas as pd
import os
from tqdm import tqdm

In [4]:
train_demand_data_2018 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2018-2019.csv')
train_demand_data_2019 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2019-2020.csv')
train_demand_data_2022 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2021-2022.csv')
train_demand_data_2023 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2022-2023.csv')

train_demand_data_precovid = pd.concat([train_demand_data_2018, train_demand_data_2019])
train_demand_data_postcovid = pd.concat([train_demand_data_2022, train_demand_data_2023])

/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_8528/3370798324.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_demand_data_2018 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2018-2019.csv')
/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_8528/3370798324.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_demand_data_2019 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2019-2020.csv')
/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_8528/3370798324.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_demand_data_2022 = pd.read_csv('../data/raw/Train_Service_Passenger_Counts_Financial_Year_2021-2022.csv')
/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_8528/3370798324.py:4: DtypeWarning: Columns (5) have mix

### Demand information

In [23]:
os.makedirs('../data/curated/train_demand', exist_ok=True)

In [5]:
def clean(train_demand_data, analysis_type):
    # clean out vline
    train_demand_data = train_demand_data[train_demand_data['Mode'] != 'VLine']

    train_demand_data['year'] = pd.to_datetime(train_demand_data['Business_Date']).dt.year

    if analysis_type == 'precovid':
        train_demand_data = train_demand_data[train_demand_data['year'] < 2020]
    elif analysis_type == 'postcovid':
        train_demand_data = train_demand_data[train_demand_data['year'] > 2021]

    # get_datetime
    train_demand_data['Departure_3hr_Time_Bracket'] = train_demand_data['Departure_Time_Scheduled'].apply(lambda x: 0 if '00:00:00' <= x < '03:00:00' else \
                                                                                                            1 if '03:00:00' <= x < '06:00:00' else \
                                                                                                                2 if '06:00:00' <= x < '09:00:00' else \
                                                                                                                    3 if '09:00:00' <= x < '12:00:00' else \
                                                                                                                        4 if '12:00:00' <= x < '15:00:00' else \
                                                                                                                            5 if '15:00:00' <= x < '18:00:00' else \
                                                                                                                                6 if '18:00:00' <= x < '21:00:00' else \
                                                                                                                                    7)
    train_demand_data['Arrival_3hr_Time_Bracket'] = train_demand_data['Arrival_Time_Scheduled'].apply(lambda x: 0 if '00:00:00' <= x < '03:00:00' else \
                                                                                                            1 if '03:00:00' <= x < '06:00:00' else \
                                                                                                                2 if '06:00:00' <= x < '09:00:00' else \
                                                                                                                    3 if '09:00:00' <= x < '12:00:00' else \
                                                                                                                        4 if '12:00:00' <= x < '15:00:00' else \
                                                                                                                            5 if '15:00:00' <= x < '18:00:00' else \
                                                                                                                                6 if '18:00:00' <= x < '21:00:00' else \
                                                                                                                                    7)
    
    return train_demand_data

In [6]:
def get_mean_daily_demand(train_demand_data):
    
    daily_deamnd = train_demand_data[['Station_Name', 'Business_Date', 'Passenger_Boardings', 'Passenger_Alightings']].groupby(['Station_Name', 'Business_Date']).agg('sum')

    mean_daily_demand = daily_deamnd.groupby('Station_Name').agg('mean').sort_values(by='Passenger_Boardings', ascending=False)

    return mean_daily_demand

In [7]:
train_demand_data_pre_covid = clean(train_demand_data_precovid, 'precovid')
mean_daily_demand_pre_covid = get_mean_daily_demand(train_demand_data_pre_covid)
mean_daily_demand_pre_covid['Total_Demand'] = mean_daily_demand_pre_covid['Passenger_Boardings'] + mean_daily_demand_pre_covid['Passenger_Alightings']
mean_daily_demand_pre_covid.to_csv('../data/curated/train_demand/mean_daily_trains_demand_pre_covid.csv')

train_demand_data_post_covid = clean(train_demand_data_postcovid, 'postcovid')
mean_daily_demand_post_covid = get_mean_daily_demand(train_demand_data_post_covid)
mean_daily_demand_post_covid['Total_Demand'] = mean_daily_demand_post_covid['Passenger_Boardings'] + mean_daily_demand_post_covid['Passenger_Alightings']
mean_daily_demand_post_covid.to_csv('../data/curated/train_demand/mean_daily_trains_demand_post_covid.csv')

/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_8528/2338728675.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_demand_data['year'] = pd.to_datetime(train_demand_data['Business_Date']).dt.year
/var/folders/tm/j_z94_gs1rncc1vngr4gz39m0000gn/T/ipykernel_8528/2338728675.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_demand_data['year'] = pd.to_datetime(train_demand_data['Business_Date']).dt.year


### Station information

In [20]:
from shapely.geometry import Point
from shapely.geometry import LineString
import geopandas as gpd

In [24]:
os.makedirs('../data/curated/stations', exist_ok=True)

In [ ]:
stations_precovid = train_demand_data_pre_covid[['Station_Name', 'Station_Longitude', 'Station_Latitude']].groupby('Station_Name').agg('first')
stations_postcovid = train_demand_data_post_covid[['Station_Name', 'Station_Longitude', 'Station_Latitude']].groupby('Station_Name').agg('first')

In [14]:
stations_precovid['geometry'] = stations_precovid.apply(lambda x: Point(x['Station_Longitude'], x['Station_Latitude']), axis=1)
stations_precovid = gpd.GeoDataFrame(stations_precovid, geometry='geometry')
stations_precovid.set_crs(epsg=7855, inplace=True)

stations_postcovid['geometry'] = stations_postcovid.apply(lambda x: Point(x['Station_Longitude'], x['Station_Latitude']), axis=1)
stations_postcovid = gpd.GeoDataFrame(stations_postcovid, geometry='geometry')
stations_postcovid.set_crs(epsg=7855, inplace=True)

,Station_Longitude,Station_Latitude,geometry
Station_Name,,,
Aircraft,144.760809,-37.866606,POINT (144.761 -37.867)
Alamein,145.079656,-37.868320,POINT (145.080 -37.868)
Albion,144.824704,-37.777653,POINT (144.825 -37.778)
Alphington,145.031255,-37.778394,POINT (145.031 -37.778)
Altona,144.829645,-37.867148,POINT (144.830 -37.867)
...,...,...,...
Williamstown Beach,144.894484,-37.863982,POINT (144.894 -37.864)
Willison,145.070298,-37.835716,POINT (145.070 -37.836)
Windsor,144.992035,-37.856053,POINT (144.992 -37.856)


In [16]:
stations_precovid.to_file('../data/curated/stations/stations_precovid.gpkg', driver='GPKG')
stations_postcovid.to_file('../data/curated/stations/stations_postcovid.gpkg', driver='GPKG')

# Line information

In [27]:
stations_precovid.index = stations_precovid['Station_Name']
stations_postcovid.index = stations_postcovid['Station_Name']

In [23]:
# get information with train lines
train_line_stations = {}
next_line_new_line = True
stations = []
with open('../data/raw/train_line.txt', 'r') as file:

    for line in file:
        if line == '\n':
            stations = []
            next_line_new_line = True
            continue
        if next_line_new_line:
            line_name = line.strip('\n')
            next_line_new_line = False
        
        stations.append(line.strip())
        train_line_stations[line_name] = stations

In [32]:
station1 = []
station2 = []
train_lines = []
lines = []


for line in train_line_stations:
    
    for i in tqdm(range(len(train_line_stations[line])-1)):
        j = 1
        try:
            stations_postcovid.loc[train_line_stations[line][i], 'geometry']
        except: 
            print('Missing initial station', train_line_stations[line][i])

        while True:
            try:
                lines.append(LineString([stations_postcovid.loc[train_line_stations[line][i], 'geometry'], stations_postcovid.loc[train_line_stations[line][i+j], 'geometry']]))
                station1.append(train_line_stations[line][i])
                station2.append(train_line_stations[line][i+j])
                train_lines.append(line)
                break
            except:
                j += 1
            if i+j >= len(train_line_stations[line]):
                break

100%|██████████| 31/31 [00:00<00:00, 14655.48it/s]


Missing initial station McKinnon
Missing initial station Glen Huntly


100%|██████████| 19/19 [00:00<00:00, 21965.76it/s]


In [47]:
# create a dataframe with the train lines
station_lines = pd.DataFrame({'station1': station1, 'station2': station2, 'line': train_lines, 'geometry': lines})

# deduplication
station_lines = station_lines.drop_duplicates()

station_lines = gpd.GeoDataFrame(station_lines)

station_lines.to_file('../data/curated/stations/station_lines.gpkg', driver='GPKG')